In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import json
import os


class SemanticMatcher:
    def __init__(self, entries):
        """
        Initialize the semantic matcher

        :param entries: Dict, {key: {description}}
        """

        # Specify a local download path
        cache_folder = os.path.expanduser("/Users/shou/Code/huggingface_models")

        # Try downloading manually first
        self.model = SentenceTransformer(
            "intfloat/multilingual-e5-large",
            cache_folder=cache_folder,
            local_files_only=False,
        )

        self.entries = entries
        # Embed all data
        self.entry_embeddings = {}
        for key, entry_info in entries.items():
            entry_text = entry_info["identification"]
            self.entry_embeddings[key] = self.model.encode(f"passage: {entry_text}")

    def match(self, query, top_k=3, threshold=0.5):
        """
        Search the queries.

        :param query: Query keywords
        :param top_k: Returns the top k most similar results
        :param threshold: Similarity threshold
        :return: Matching entries and their similarities
        """
        # Generate an embedding vector for the query
        query_with_prefix = f"query: {query}"
        query_embedding = self.model.encode(query_with_prefix)

        # Calculating similarity
        similarities = {}
        for key, entry_embedding in self.entry_embeddings.items():
            similarity = np.dot(query_embedding, entry_embedding) / (
                np.linalg.norm(query_embedding) * np.linalg.norm(entry_embedding)
            )
            similarities[key] = similarity

        # Sort by similarity
        sorted_matches = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

        # Filter and return results
        filtered_matches = [
            (key, similarity)
            for key, similarity in sorted_matches
            if similarity >= threshold
        ]

        return filtered_matches[:top_k]

In [40]:
# Input bird identifications
with open("ebird_data.json",'r', encoding='UTF-8') as f:
     entries = json.load(f)

# Encode the passages
matcher = SemanticMatcher(entries)

In [54]:
# Queries
test_queries = ["中型でやや細身のフクロウで、羽角が長い。北半球に広く分布する。北アメリカの個体はやや色が濃く、模様が粗く、顔が橙黄色で目が黄色い。ユーラシア大陸の個体群は、全体的に色が薄く、特に顔の色が薄く、赤みがかった目をしている。夜行性で、夕暮れや夜明けに採餌する姿がまれに見られる。草が茂った雑木林や針葉樹林など、ねぐらのための植生が密な環境と、狩りをするための開けた場所が混在するところで見られる。珍しく、めったに見られないが、冬に数十羽でねぐらに集まることがある（幸運にもねぐらを見つけたら、彼らを妨害しないこと）。コミミズクと比較できる。典型的な鳴き声は低く「ウーウー」と繰り返し、また、猫のような叫び声や犬のような吠え声など、幅広い声を出す。"]
for query in test_queries:
    print(f"\nSearch: {query}")
    results = matcher.match(query)

    for key, similarity in results:
        print(f"Matched: {key}, Similarity: {similarity:.4f}")
        print("Detail:", entries[key]["binomialName"], entries[key]["url"])


Search: 中型でやや細身のフクロウで、羽角が長い。北半球に広く分布する。北アメリカの個体はやや色が濃く、模様が粗く、顔が橙黄色で目が黄色い。ユーラシア大陸の個体群は、全体的に色が薄く、特に顔の色が薄く、赤みがかった目をしている。夜行性で、夕暮れや夜明けに採餌する姿がまれに見られる。草が茂った雑木林や針葉樹林など、ねぐらのための植生が密な環境と、狩りをするための開けた場所が混在するところで見られる。珍しく、めったに見られないが、冬に数十羽でねぐらに集まることがある（幸運にもねぐらを見つけたら、彼らを妨害しないこと）。コミミズクと比較できる。典型的な鳴き声は低く「ウーウー」と繰り返し、また、猫のような叫び声や犬のような吠え声など、幅広い声を出す。
Matched: Long-eared Owl, Similarity: 0.8429
Detail: Asio otus https://ebird.org/species/loeowl/JP-13
Matched: Eyebrowed Thrush, Similarity: 0.8218
Detail: Turdus obscurus https://ebird.org/species/eyethr/JP-13
Matched: Wilson's Storm-Petrel, Similarity: 0.8171
Detail: Oceanites oceanicus https://ebird.org/species/wispet/JP-13
